# Feature value extraction from melanoma images

In [9]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
from  pandas.core.frame import DataFrame
import pandas as pd

In [10]:
### read image file
filename = 'test.jpg'
img_ori = read_image(filename)

### remove hair (closing operation)
(img,closing3) = remove_hair_closing(img_ori)
img_binary = image_binarization(closing3)

### remove Removing peripheral irrelevant areas
path_values_0 = []
path_values_1 = []
path_values_2 = []
path_values_3 = []

path_0 = []
path_1 = []
path_2 = []
path_3 = []

diagonal_direction(img_binary,path_values_0,path_0,0)
diagonal_direction(img_binary,path_values_1,path_1,1)
diagonal_direction(img_binary,path_values_2,path_2,2)
diagonal_direction(img_binary,path_values_3,path_3,3)

border_junction(path_values_0)
if border_junction(path_values_0)>0 and\
   border_junction(path_values_1)>0 and\
   border_junction(path_values_2)>0 and\
   border_junction(path_values_3)>0:
    (Y1,X1) = path_0[-border_junction(path_values_0)] 
    (Y3,X3) = path_3[-border_junction(path_values_3)]
    (Y4,X4) = path_1[-border_junction(path_values_1)] #右上
    (Y5,X5) = path_2[-border_junction(path_values_2)] #左下
    closing4_1 = closing3[Y1:Y3, X1:X3]
    closing4_2 = closing3[Y4:Y5, X5:X4]
    if closing4_1.shape[0]*closing4_1.shape[1] < closing4_2.shape[0]*closing4_2.shape[1]:
        closing3 = closing4_1
    else:
        closing3 = closing4_2
        
### remove hair again(opening operation)
img_binary = image_binarization(closing3)
opening = remove_hair_opening(img_binary)

### calculate Irregularity
(max_con,l_im) = find_max_contours(img,opening)
(area_rect,box,img_copy)=draw_rectangle(max_con,l_im)
#print('Rectangle area:', area_rect)

(area_ell,img_copy)=draw_ellipse(l_im,img_copy)
#print('ellipse area:', area_ell)

(area_m,ae_diff,ar_diff) = calculate_irregularity_value(l_im,area_ell,area_rect) 
#print('The area of the melanoma:', area_m)
#print('Difference between melanoma and ellipse area:', ae_diff) 
#print('Difference between melanoma and rectangle area:', ar_diff)


### calculate the proportion of melanoma symptoms in the image
area_sp = image_area(img_ori)
#print('Image area:', area_sp)

Proportion = tumor_proportion(area_m,area_sp)
#print('Proportion of the melanoma in the image:', Proportion)

### trim the melanoma symptoms 
masked_image = mask_symptom_images(closing3,box)
angle = calculate_angle( box[0][0],  box[0][1],  box[1][0],  box[1][1])
rotated = rotate_image(masked_image, angle-90)
rotated_img_bin = rotate_image((img_binary*255).astype(np.uint8), angle-90)

crop_img, crop_img_bin = crop_rotated_image(rotated,rotated_img_bin)

### calculate symmetry feature
sym = calculate_symmetry(crop_img_bin)

### calaulate hsv and rgb channel features
stathsv = calculate_hsv(crop_img)
statrgb = calculate_rgb(crop_img)

### calculate Gray-Level Co-occurrence Matrix
(contrast,dissimilarity,homogeneity,energy,correlation) = calculate_glcm(crop_img)



In [11]:
### Organize all the recently generated feature values.

h_mean = stathsv[0]
s_mean = stathsv[1]
v_mean = stathsv[2]
h_std = stathsv[3]
s_std = stathsv[4]
v_std = stathsv[5]
h_skw = stathsv[6]
s_skw = stathsv[7]
v_skw = stathsv[8]

r_mean = statrgb[0]
g_mean = statrgb[1]
b_mean = statrgb[2]
r_std = statrgb[3]
g_std = statrgb[4]
b_std = statrgb[5]
r_skw = statrgb[6]
g_skw = statrgb[7]
b_skw = statrgb[8]
contrast0 = contrast[0][0]
contrast45 = contrast[0][1]
contrast90 =  contrast[0][2]
contrast135 =  contrast[0][3]
dissimilarity0 = dissimilarity[0][0]
dissimilarity45 = dissimilarity[0][1]
dissimilarity90 =  dissimilarity[0][2]
dissimilarity135 =  dissimilarity[0][3]
homogeneity0 = homogeneity[0][0]
homogeneity45 = homogeneity[0][1]
homogeneity90 =  homogeneity[0][2]
homogeneity135 =  homogeneity[0][3]
energy0 = energy[0][0]
energy45 = energy[0][1]
energy90 =  energy[0][2]
energy135 =  energy[0][3]
correlation0 = correlation[0][0]
correlation45 = correlation[0][1]
correlation90 =  correlation[0][2]
correlation135 =  correlation[0][3]

charac = [None] * 42

charac[0] = ae_diff
charac[1] = ar_diff
charac[2] = Proportion 
charac[3] = h_mean
charac[4] = h_std
charac[5] = h_skw 
charac[6] = s_mean 
charac[7] = s_std 
charac[8] = s_skw
charac[9] = v_mean 
charac[10] = v_std 
charac[11] = v_skw 
charac[12] = contrast0 
charac[13] = contrast45 
charac[14] = contrast90 
charac[15] = contrast135 
charac[16] = dissimilarity0 
charac[17] = dissimilarity45
charac[18] = dissimilarity90
charac[19] = dissimilarity135
charac[20] = homogeneity0 
charac[21] = homogeneity45 
charac[22] = homogeneity90
charac[23] = homogeneity135
charac[24] = energy0
charac[25] = energy45
charac[26] = energy90
charac[27] = energy135
charac[28] = correlation0
charac[29] = correlation45
charac[30] = correlation90
charac[31] = correlation135
charac[32] = sym
charac[33] = r_mean
charac[34] = r_std
charac[35] = r_skw 
charac[36] = g_mean 
charac[37] = g_std 
charac[38] = g_skw
charac[39] = b_mean 
charac[40] = b_std 
charac[41] = b_skw 

# features are written to excel

In [12]:
charac_df = np.array(charac).reshape(1,42)

In [13]:
column_name = ['ae_diff', 'ar_diff','Proportion', 'h_mean' , 'h_std', 'h_skw', 's_mean', 's_std', 's_skw', 'v_mean', 'v_std', 
              'v_skw', 'contrast0', 'contrast45', 'contrast90', 'contrast135', 'dissimilarity0', 'dissimilarity45', 'dissimilarity90', 
              'dissimilarity135', 'homogeneity0', 'homogeneity45', 'homogeneity90','homogeneity135', 'energy0', 'energy45',
              'energy90', 'energy135', 'correlation0', 'correlation45', 'correlation90', 'correlation135','sym',
               'r_mean' , 'r_std', 'r_skw', 'g_mean', 'g_std', 'g_skw', 'b_mean', 'b_std', 'b_skw']
DataFrame(charac_df,columns=column_name)

,ae_diff,ar_diff,Proportion,h_mean,h_std,h_skw,s_mean,s_std,s_skw,v_mean,...,sym,r_mean,r_std,r_skw,g_mean,g_std,g_skw,b_mean,b_std,b_skw
0,0.108192,0.859646,0.312199,11.272236,26.228474,47.75753,123.734542,46.893015,52.05118,151.86082,...,0.125271,151.860178,54.331165,61.113307,100.44476,57.225509,62.74474,83.901044,51.758773,57.042627


In [14]:
writer = pd.ExcelWriter(r'melanoma_features.xlsx')
DataFrame(charac_df,columns=column_name).to_excel(writer)  
writer._save()